# Troubleshooting and Debugging

## Objectives

By the end of this session, you should be able to:

* Use print statements in CUDA kernels to aid your understanding of what happens inside them.
* Use the CUDA simulator to debug CUDA kernels using the Python debugger.
* Use cuda-memcheck to pinpoint invalid memory accesses in CUDA kernels.

## Note about the Terminal

Debugging is an important part of programming.  Unfortuntely, it is pretty difficult to debug CUDA kernels directly in the Jupyter notebook for a variety of reasons, so this notebook will show terminal commands by executing Jupyter notebook cells using the shell.  These shell commands will appear in notebook cells with the command line prefixed by `!`. When applying the debug methods described in this notebook, you will likely run the commands in the terminal directly.

## Printing

A common debugging strategy is printing to the console.  Numba supports printing from CUDA kernels, with some restrictions.  Note that output printed from a CUDA kernel will not be captured by Jupyter, so you will need to debug with a script you can run from the terminal.

Let's look at a CUDA kernel with a bug:

In [ ]:
! cat debug/ex1.py

When we run this code to histogram 50 values, we see the histogram is not getting 50 entries: 

In [ ]:
! python debug/ex1.py

*(You might have already spotted the mistake, but let's pretend we don't know the answer.)*

We hypothesize that maybe a bin calculation error is causing many of the histogram entries to appear out of range.  Let's add some printing around the `if` statement to show us what is going on:

In [ ]:
! cat debug/ex1a.py

This kernel will print every value and bin number it calculates.  Looking at one of the print statements, we see that `print` supports constant strings, and scalar values:

``` python
print('in range', x[i], bin_number)
```

String substitution (using C printf syntax or the newer `format()` syntax) is not supported.  If we run this script we see:

In [ ]:
! python debug/ex1a.py

Scanning down that output, we see that all 50 values should be in range.  Clearly we have some kind of race condition updating the histogram.  In fact, the culprit line is:

``` python
histogram_out[bin_number] += 1
```

which should be (as you may have seen in a previous exercise)

``` python
cuda.atomic.add(histogram_out, bin_number, 1)
```

### CUDA Simulator

Back in the early days of CUDA, `nvcc` had an "emulator" mode that would execute CUDA code on the CPU for debugging.  That functionality was dropped in later CUDA releases after `cuda-gdb` was created.  There isn't a debugger for CUDA+Python, so Numba includes a "CUDA simulator" in Numba that runs your CUDA code with the Python interpreter on the host CPU.  This allows you to debug the logic of your code using Python modules and functions that would otherwise be not allowed by the compile.

A very common use case is to start the Python debugger inside one thread of a CUDA kernel:
``` python
import numpy as np

from numba import cuda

@cuda.jit
def histogram(x, xmin, xmax, histogram_out):
    nbins = histogram_out.shape[0]
    bin_width = (xmax - xmin) / nbins

    start = cuda.grid(1)
    stride = cuda.gridsize(1)

    ### DEBUG FIRST THREAD
    if start == 0:
        from pdb import set_trace; set_trace()
    ###

    for i in range(start, x.shape[0], stride):
        bin_number = np.int32((x[i] + xmin)/bin_width)

        if bin_number >= 0 and bin_number < histogram_out.shape[0]:
            cuda.atomic.add(histogram_out, bin_number, 1)

x = np.random.normal(size=50, loc=0, scale=1).astype(np.float32)
xmin = np.float32(-4.0)
xmax = np.float32(4.0)
histogram_out = np.zeros(shape=10, dtype=np.int32)

histogram[64, 64](x, xmin, xmax, histogram_out)

print('input count:', x.shape[0])
print('histogram:', histogram_out)
print('count:', histogram_out.sum())
```

This code allows a debug session like the following to take place:
```
(gtc2017) 0179-sseibert:gtc2017-numba sseibert$ NUMBA_ENABLE_CUDASIM=1 python debug/ex2.py
> /Users/sseibert/continuum/conferences/gtc2017-numba/debug/ex2.py(18)histogram()
-> for i in range(start, x.shape[0], stride):
(Pdb) n
> /Users/sseibert/continuum/conferences/gtc2017-numba/debug/ex2.py(19)histogram()
-> bin_number = np.int32((x[i] + xmin)/bin_width)
(Pdb) n
> /Users/sseibert/continuum/conferences/gtc2017-numba/debug/ex2.py(21)histogram()
-> if bin_number >= 0 and bin_number < histogram_out.shape[0]:
(Pdb) p bin_number, x[i]
(-6, -1.4435024)
(Pdb) p x[i], xmin, bin_width
(-1.4435024, -4.0, 0.80000000000000004)
(Pdb) p (x[i] - xmin) / bin_width
3.1956219673156738
(Pdb) q
```

### CUDA Memcheck

Another common error occurs when a CUDA kernel has an invalid memory access, typically caused by running off the end of an array.  The full CUDA toolkit from NVIDIA (not the `cudatoolkit` conda package) contain a utility called `cuda-memcheck` that can check for a wide range of memory access mistakes in CUDA code.

Let's debug the following code:

In [ ]:
! cat debug/ex3.py

In [ ]:
! cuda-memcheck python debug/ex3.py

The output of `cuda-memcheck` is clearly showing a problem with our histogram function:
```
========= Invalid __global__ write of size 4
=========     at 0x00000548 in cudapy::__main__::histogram$241(Array<float, int=1, C, mutable, aligned>, float, float, Array<int, int=1, C, mutable, aligned>)
```
But we don't know which line it is.  To get better error information, we can turn "debug" mode on when compiling the kernel, by changing the kernel to look like this:
``` python
@cuda.jit(debug=True)
def histogram(x, xmin, xmax, histogram_out):
    nbins = histogram_out.shape[0]
```

In [ ]:
! cuda-memcheck python debug/ex3a.py

Now we get an error message that includes a source file and line number: `ex3a.py:17`.

In [ ]:
! cat -n debug/ex3a.py | grep -C 2 "17"

At this point, we might realize that our if statement incorrectly has an `or` instead of an `and`.

`cuda-memcheck` has different modes for detecting different kinds of problems (similar to `valgrind` for debugging CPU memory access errors).  Take a look at the documentation for more information: http://docs.nvidia.com/cuda/cuda-memcheck/